In [ ]:
import importlib
import torch
import Data_Prep
importlib.reload(Data_Prep)
from Data_Prep import Data_Prep, Player_IO
import Prep_Map
importlib.reload(Prep_Map)
import Output_Map
importlib.reload(Output_Map)
import Player_Dataset
importlib.reload(Player_Dataset)
from Player_Dataset import Player_Dataset, Create_Test_Train_Datasets
import Player_Model
importlib.reload(Player_Model)
from Player_Model import RNN_Model
from torch.optim import lr_scheduler
import Model_Train
importlib.reload(Model_Train)
from Model_Train import trainAndGraph
from Constants import device, DEFAULT_HIDDEN_SIZE_PITCHER, DEFAULT_NUM_LAYERS_PITCHER

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
batch_size = 200
num_epochs = 50
num_layers = DEFAULT_NUM_LAYERS_PITCHER
hidden_size = DEFAULT_HIDDEN_SIZE_PITCHER

Experimental Pretraining

In [ ]:
data_prep = Data_Prep(Prep_Map.experimental_prep_map, Output_Map.base_output_map)
pitcher_io_list = data_prep.Generate_IO_Pitchers("WHERE lastMLBSeason<? AND signingYear<? AND isPitcher=?", (2025,2015,1), use_cutoff=True)
train_dataset, test_dataset = Create_Test_Train_Datasets(pitcher_io_list, 0.25, 0)

In [ ]:
importlib.reload(Player_Model)
from Player_Model import RNN_Model
importlib.reload(Model_Train)
from Model_Train import trainAndGraph

mutators = data_prep.Generate_Pitching_Mutators(batch_size, Player_IO.GetMaxLength(pitcher_io_list))

network = RNN_Model(train_dataset.get_input_size(), num_layers, hidden_size, mutators, data_prep=data_prep, is_hitter=False)
network = network.to(device)

trainAndGraph(network, 
              train_dataset, 
              test_dataset,
              batch_size,
              num_epochs = num_epochs, 
              logging_interval=10, 
              early_stopping_cutoff=2000, 
              should_output=True, 
              is_hitter=False, 
              model_name="Models/default_experimental_pitcher", 
              save_last=True,
              elements_to_save=[0])

Test

In [ ]:
import torch

pitlvlstat_means : torch.Tensor = getattr(data_prep, "__pitlvlstat_means").to(device)
pitlvlstat_devs : torch.Tensor = getattr(data_prep, "__pitlvlstat_devs").to(device)
pitpt_means : torch.Tensor = getattr(data_prep, "__pitlvlpt_means").to(device)
pitpt_devs : torch.Tensor = getattr(data_prep, "__pitlvlpt_devs").to(device)

print(pitpt_means)
print(pitpt_devs)

torch.set_printoptions(precision=8, sci_mode=False)
torch.set_printoptions(
    precision=6,
    sci_mode=False,
    linewidth=500,
    edgeitems=6,
)

for pitcher in pitcher_io_list:
    if pitcher.player.mlbId != 608648:
        continue
    
    data = pitcher.input.to(device)
    length = torch.tensor([pitcher.length]).to(device)
    output_war, output_warregression, output_level, output_pa, output_yearStats, output_yearPos, output_mlbValue, output_pt = network(data.unsqueeze(0), length)
    
    dates = pitcher.dates
    output_yearStats = output_yearStats.squeeze(0)
    actual_pt = pitcher.pt_year_output.squeeze(-1).to(device)
    output_pt = output_pt.squeeze(0)
    output_pt = output_pt.reshape((output_pt.size(0), actual_pt.size(1), actual_pt.size(2)))
    print(output_pt.shape)
    print(actual_pt.shape)
    for i in range(dates.size(0)):
        for j in range(8):
            output_pt[i,j] = (output_pt[i,j] * pitpt_devs) + pitpt_means
            actual_pt[i,j] = (actual_pt[i,j] * pitpt_devs) + pitpt_means
        print(f"Act:  {dates[i,1].item():4d}-{dates[i,2].item()} : MLB={actual_pt[i,0,0].item():3.0f}:{actual_pt[i,0,1].item():3.0f},AAA={actual_pt[i,1,0].item():3.0f}:{actual_pt[i,1,1].item():3.0f},AA={actual_pt[i,2,0].item():3.0f}:{actual_pt[i,2,1].item():3.0f},A+={actual_pt[i,3,0].item():3.0f}:{actual_pt[i,3,1].item():3.0f},A={actual_pt[i,4,0].item():3.0f}:{actual_pt[i,4,1].item():3.0f},A-={actual_pt[i,5,0].item():3.0f}:{actual_pt[i,5,1].item():3.0f},Rk={actual_pt[i,6,0].item():3.0f}:{actual_pt[i,6,1].item():3.0f},DSL={actual_pt[i,7,0].item():3.0f}:{actual_pt[i,7,1].item():3.0f}")
        print(f"Pred: {dates[i,1].item():4d}-{dates[i,2].item()} : MLB={output_pt[i,0,0].item():3.0f}:{output_pt[i,0,1].item():3.0f},AAA={output_pt[i,1,0].item():3.0f}:{output_pt[i,1,1].item():3.0f},AA={output_pt[i,2,0].item():3.0f}:{output_pt[i,2,1].item():3.0f},A+={output_pt[i,3,0].item():3.0f}:{output_pt[i,3,1].item():3.0f},A={output_pt[i,4,0].item():3.0f}:{output_pt[i,4,1].item():3.0f},A-={output_pt[i,5,0].item():3.0f}:{output_pt[i,5,1].item():3.0f},Rk={output_pt[i,6,0].item():3.0f}:{output_pt[i,6,1].item():3.0f},DSL={output_pt[i,7,0].item():3.0f}:{output_pt[i,7,1].item():3.0f}\n")
    

Base Pretraining

In [ ]:
data_prep = Data_Prep(Prep_Map.base_prep_map, Output_Map.base_output_map)
pitcher_io_list = data_prep.Generate_IO_Pitchers("WHERE lastMLBSeason<? AND signingYear<? AND isPitcher=?", (2025,2015,1), use_cutoff=True)
train_dataset, test_dataset = Create_Test_Train_Datasets(pitcher_io_list, 0.25, 0)

In [ ]:
mutators = data_prep.Generate_Pitching_Mutators(batch_size, Player_IO.GetMaxLength(pitcher_io_list))

network = RNN_Model(train_dataset.get_input_size(), num_layers, hidden_size, mutators, data_prep=data_prep, is_hitter=False)
network = network.to(device)

trainAndGraph(network, 
              train_dataset, 
              test_dataset,
              batch_size,
              num_epochs = num_epochs, 
              logging_interval=10, 
              early_stopping_cutoff=2000, 
              should_output=True, 
              is_hitter=False, 
              model_name="Models/default_pitcher", 
              save_last=True,
              elements_to_save=[0])

Stats Only Pretraining

In [ ]:
data_prep = Data_Prep(Prep_Map.statsonly_prep_map, Output_Map.base_output_map)
pitcher_io_list = data_prep.Generate_IO_Pitchers("WHERE lastMLBSeason<? AND signingYear<? AND isPitcher=?", (2025,2015,1), use_cutoff=True)
train_dataset, test_dataset = Create_Test_Train_Datasets(pitcher_io_list, 0.25, 0)

In [ ]:
mutators = data_prep.Generate_Pitching_Mutators(batch_size, Player_IO.GetMaxLength(pitcher_io_list))
network = RNN_Model(train_dataset.get_input_size(), num_layers, hidden_size, mutators, data_prep=data_prep, is_hitter=False)
network = network.to(device)

trainAndGraph(network, 
              train_dataset, 
              test_dataset,
              batch_size,
              num_epochs = num_epochs, 
              logging_interval=10, 
              early_stopping_cutoff=2000, 
              should_output=True, 
              model_name="Models/default_statsonly_pitcher", 
              save_last=True,
              elements_to_save=[0])